<span style="color:red">This is just to share the code with you - we're probably going to move this data reading to a function, such that we don't have to repeat the code in every notebook for each task.</span>.

In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4
plt.rcParams['axes.axisbelow'] = True

## Read data for the 24-bus IEEE test system

<em>(this is at the moment a copy-paste from Julia's & Markus' work in course 46750)</em>

<span style="color:red">Ensure that the input data to be used is the same for this assignment.</span>.

In [3]:
#Assignment variables applicable across all tasks
n_bus = 24
wind_MWp = 300
S_base_3ph = 100

#path is the path to the folder containing the files for the assignment
path = "data/"

gen_costs = pd.read_excel(path + 'gen_costs.xlsx')['C ($/MWh)'].to_frame()
gen_costs['C (DKK/MWh)'] = gen_costs['C ($/MWh)'].values * 7.03 #USD to DKK

gen_data = pd.read_excel(path + 'gen_data.xlsx')
line_data = pd.read_excel(path + 'line_data.xlsx')

system_demand = pd.read_excel(path + 'system_demand.xlsx')
load_distribution = pd.read_excel(path + 'load_distribution.xlsx')

# Wind data processing - see the notebook for visualization
wind_data_raw = []
for i in range(6):
    wt = pd.read_csv(path + 'wind %d.out' % (i + 1))
    wt = wt.drop(columns = ['Unnamed: 0']) #dropping the unnecessary index column
    wt['Mean'] = wt.mean(numeric_only=True, axis=1)
    wind_data_raw.append(wt)

wind_data = pd.DataFrame(index = wind_data_raw[0].index, data = wind_data_raw[0]['Mean'].values * wind_MWp, columns=['WF1 MW'])

for i in range(1,6):
    wind_data['WF%d MW' % (i + 1)] = wind_data_raw[i]['Mean'].values * wind_MWp #from normalized to MW values

wind_data = wind_data[12:36].reset_index(drop=True) #selected data is from hours 13 to and including 36 (0-indexed: 12 to and including 35)


#Filling out branch susceptance matrix and adjusting branch data

# Important:
# Please consider that the capacity of the transmission lines connecting the 
# node pairs (15, 21), (14, 16) and (13, 23) is reduced to 400 MW, 250 MW and 250 MW, respectively
line_data.loc[(line_data['From'] == 15) & (line_data['To'] == 21), 'Capacity MVA'] = 400
line_data.loc[(line_data['From'] == 14) & (line_data['To'] == 16), 'Capacity MVA'] = 250
line_data.loc[(line_data['From'] == 13) & (line_data['To'] == 23), 'Capacity MVA'] = 250
line_data['Susceptance pu'] = 1 / line_data['Reactance pu']
line_data['Capacity pu'] = line_data['Capacity MVA'] / S_base_3ph

branch_matrix = np.zeros((n_bus, n_bus))

for n in range(1, n_bus + 1):
    branch_matrix[n - 1][n - 1] = line_data.loc[(line_data['From'] == n) | (line_data['To'] == n), 'Susceptance pu'].sum()
    for k in range(n, n_bus + 1): #start from n
        if k != n:
            branch_matrix[n - 1][k - 1] = -1 * (line_data.loc[(line_data['From'] == n) & (line_data['To'] == k), 'Susceptance pu'].sum())
            branch_matrix[k - 1][n - 1] = branch_matrix[n - 1][k - 1] #symmetric
    

#These dictionarys return the 0-indexed indices of generators or wind farms located at the node n

#for example, gens_map.get(0) will return the list [0] because generator 1 is placed at node 1
#gens_map.get(14) will return [4, 5] because generators 5 and 6 are at node 15

#if no wind farms or generators are located at node n, it will return an empty list

wf_nodes = [2, 4, 6, 15, 20, 22]

gens_map = {}

for n in range(1, n_bus + 1):
    gens_map[n - 1] = (gen_data['Unit #'][gen_data['Node'] == n] - 1).tolist()

wf_map = {}

for n in range(n_bus):
    wf_map[n] = np.where(np.array(wf_nodes) == n)[0].tolist()

In [7]:
gens_map[22]

[10, 11]